# Import Dependencies

In [1]:
import os
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, random_split, DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from collections import OrderedDict
from tqdm import tqdm
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch import optim
import time
from torch.autograd import Variable
%matplotlib inline

In [2]:
SEED = 1

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# Load Data

In [3]:
data_dir = "mliii-assignment2" # Path to data directory
labels = pd.read_csv(os.path.join(data_dir, 'labels.csv'))
assert(len(os.listdir(os.path.join(data_dir, 'train'))) == len(labels))

In [4]:
le = LabelEncoder()
labels.breed = le.fit_transform(labels.breed)
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,19
1,001513dfcb2ffafc82cccf4d8bbaba97,37
2,001cdf01b096e06d78e9e5112d419397,85
3,00214f311d5d2247d5dfe4fe24b2303d,15
4,0021f9ceb3235effd7fcde7f7538ed62,49


In [5]:
X = labels.id
y = labels.breed

X_train, X_valid, y_train, y_valid = train_test_split(X, y,test_size=0.4, random_state=SEED, stratify=y)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=SEED, stratify=y_valid)

In [6]:
from torchvision.transforms import Resize, ToTensor, Normalize

means = np.load('means.npy')
stds = np.load('stds.npy')
class Dataset_Interpreter(Dataset):
    def __init__(self, data_path, file_names, labels=None):
        self.data_path = data_path
        self.file_names = file_names
        self.labels = labels
        self.transforms = transforms
        self.resize = Resize((224,224))
        self.tensor = ToTensor(),
        self.normalize = Normalize(means, stds)
        self.trans = transforms.Compose([
                transforms.ToTensor()])
        
    def __len__(self):
        return (len(self.file_names))
    
    def __getitem__(self, idx):
        img_name = f'{self.file_names.iloc[idx]}.jpg'
        full_address = os.path.join(self.data_path, img_name)
        image = Image.open(full_address)
        label = self.labels.iloc[idx]
        image = self.resize(image)
        image = self.trans(image)
        image = self.normalize(image)
#         if self.transforms is not None:
#             image = self.transforms(image)
        
        return image, label

In [7]:
def plot_images(images):

    n_images = len(images)

    rows = int(np.sqrt(n_images))
    cols = int(np.sqrt(n_images))

    fig = plt.figure(figsize=(20,10))
    for i in range(rows*cols):
        ax = fig.add_subplot(rows, cols, i+1)
        ax.set_title(f'{le.inverse_transform([images[i][1]])}')
        ax.imshow(np.array(images[i][0]))
        ax.axis('off')
N_IMAGES = 9

train_data = Dataset_Interpreter(data_path=data_dir+'/train/', file_names=X_train, labels=y_train)
valid_data = Dataset_Interpreter(data_path=data_dir+'/valid/', file_names=X_valid, labels=y_valid)
test_data = Dataset_Interpreter(data_path=data_dir+'/test/', file_names=X_test, labels=y_test)
images = [(image, label) for image, label in [train_data[i] for i in range(N_IMAGES)]] 
#plot_images(images)

In [137]:
train_data[0]

(tensor([[[-1.8409, -1.8399, -1.8394,  ..., -1.8326, -1.8328, -1.8332],
          [-1.8406, -1.8398, -1.8392,  ..., -1.8327, -1.8329, -1.8333],
          [-1.8401, -1.8395, -1.8388,  ..., -1.8328, -1.8330, -1.8334],
          ...,
          [-1.8384, -1.8432, -1.8436,  ..., -1.8348, -1.8345, -1.8346],
          [-1.8360, -1.8413, -1.8439,  ..., -1.8350, -1.8342, -1.8346],
          [-1.8352, -1.8380, -1.8430,  ..., -1.8348, -1.8341, -1.8348]],
 
         [[-1.7839, -1.7830, -1.7824,  ..., -1.7755, -1.7758, -1.7762],
          [-1.7836, -1.7828, -1.7822,  ..., -1.7756, -1.7760, -1.7763],
          [-1.7831, -1.7825, -1.7818,  ..., -1.7757, -1.7761, -1.7765],
          ...,
          [-1.7813, -1.7862, -1.7867,  ..., -1.7779, -1.7776, -1.7777],
          [-1.7788, -1.7844, -1.7870,  ..., -1.7782, -1.7772, -1.7777],
          [-1.7780, -1.7809, -1.7862,  ..., -1.7780, -1.7772, -1.7779]],
 
         [[-1.5214, -1.5203, -1.5196,  ..., -1.5120, -1.5122, -1.5125],
          [-1.5211, -1.5202,

In [76]:
len(train_data)

6133

channel_1_mean = np.mean([train_data[x][0][:,:,0] for x in range(0, len(train_data))])

channel1 = [train_data[x][0][:,:,0] for x in range(0, len(train_data))]

channel2 = [train_data[x][0][:,:,1] for x in range(0, len(train_data))]
channel3 = [train_data[x][0][:,:,2] for x in range(0, len(train_data))]

channel_2_mean = np.mean(channel2)
channel_3_mean = np.mean(channel3)
channel_1_std = np.std(channel1)
channel_2_std = np.std(channel2)
channel_3_std = np.std(channel3)

means = np.array([channel_1_mean, channel_2_mean, channel_3_mean])
stds = np.array([channel_1_std, channel_2_std, channel_3_std])
np.save('stds', stds)
np.save('means', means)

# Data Preprocessing

In [138]:
# # Data augmentation and normalization for training
# # Just normalization for validation
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize(means, stds)
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize(means, stds)
#     ]),
# }

# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
#                                           data_transforms[x])
#                   for x in ['train', 'val']}
# dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
#                                              shuffle=True, num_workers=4)
#               for x in ['train', 'val']}
# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
# class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Create Model

In [139]:
# Create model here 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

model_ft = models.resnet50(weights='IMAGENET1K_V2')
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 120)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [146]:
loss_values= []
batch_size = 100
# Pytorch train and test sets
#train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)

# data loader
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = False)
#test_loader = DataLoader(test_numpy, batch_size = batch_size, shuffle = False)

def train_model(model, optimizer, data_loader, loss_module, num_epochs=100):
    # Set model to train mode
    model.train()

    # Training loop
    for epoch in tqdm(range(num_epochs)):
        for data_inputs, data_labels in data_loader:

            ## Step 1: Move input data to device (only strictly necessary if we use GPU)
            data_inputs = data_inputs.to(device)
            data_labels = np.array(data_labels)
            data_labels = torch.from_numpy(data_labels)
            data_labels = data_labels.to(device)

            ## Step 2: Run the model on the input data
            preds = model(data_inputs)
            preds = preds.squeeze(dim=1) # Output is [Batch size, 1], but we want [Batch size]

            ## Step 3: Calculate the loss
            loss = loss_module(preds, data_labels)
            loss_values.append(loss)
            ## Step 4: Perform backpropagation
            # Before calculating the gradients, we need to ensure that they are all zero.
            # The gradients would not be overwritten, but actually added to the existing ones.
            optimizer.zero_grad()
            # Perform backpropagation
            loss.backward()

            ## Step 5: Update the parameters
            optimizer.step()
            
train_model(model_ft, optimizer_ft, train_loader, criterion)
#plt.plot(loss_values)

  6%|▌         | 6/100 [03:54<1:01:13, 39.08s/it]


TypeError: int() argument must be a string, a bytes-like object or a number, not 'Image'

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

# Submission CSV

In [ ]:
import pandas as pd

def generate_submission(predictions, sample_submission_path, output_path):
    """
    Generate a Kaggle submission file based on the given predictions.

    Parameters:
    - predictions: A dictionary with image ids as keys and a list of 120 probabilities as values.
    - sample_submission_path: Path to the provided sample submission file.
    - output_path: Path to save the generated submission file.
    """
    # Load the sample submission
    sample_submission = pd.read_csv(sample_submission_path)
    
    # Replace the sample probabilities with the actual predictions
    for image_id, probs in predictions.items():
        sample_submission.loc[sample_submission['id'] == image_id, sample_submission.columns[1:]] = probs

    # Save the modified sample submission as the final submission
    sample_submission.to_csv(output_path, index=False)

# Example usage
predictions = {
    '000621fb3cbb32d8935728e48679680e': [0.01, 0.02, ...],  # Replace with actual probabilities
    # ... add more predictions
}
generate_submission(predictions, data_dir + '/sample_submission.csv', 'my_submission.csv')